 # MTH8408 : Méthodes d'optimisation et contrôle optimal
 ## Laboratoire 3: Optimisation sans contraintes et méthodes itératives
Tangi Migot et Paul Raynaud

In [1]:
using LinearAlgebra, NLPModels, Printf

In [2]:
#Test problem:
using ADNLPModels
fH(x) = (x[2]+x[1].^2-11).^2+(x[1]+x[2].^2-7).^2
x0H = [10., 20.]
himmelblau = ADNLPModel(fH, x0H)

problem2 = ADNLPModel(x->-x[1]^2, ones(3))

roz(x) = 100 *  (x[2] - x[1]^2)^2 + (x[1] - 1.0)^2
rosenbrock = ADNLPModel(roz, [-1.2, 1.0])

f(x) = x[1]^2 * (2*x[1] - 3) - 6*x[1]*x[2] * (x[1] - x[2] - 1)
pb_du_cours = ADNLPModel(f, [-1.001, -1.001]) #ou [1.5, .5] ou [.5, .5]

ADNLPModel - Model with automatic differentiation backend ADModelBackend{
  ForwardDiffADGradient,
  ForwardDiffADHvprod,
  EmptyADbackend,
  EmptyADbackend,
  EmptyADbackend,
  ForwardDiffADHessian,
  EmptyADbackend,
}
  Problem name: Generic
   All variables: ████████████████████ 2      All constraints: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            free: ████████████████████ 2                 free: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                lower: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                upper: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
         low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0              low/upp: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
           fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0                fixed: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
          infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0               infeas: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
            nnzh: (  0.00% sparsity)   3               linear: ⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅⋅ 0     
                            

## Commentaires sur Julia

Quelques commentaires sur des morceaux de codes que vous avez vu:
- les structures, exemple [GenericExecutionStats](https://github.com/JuliaSmoothOptimizers/SolverCore.jl/blob/0091f437a26a27ac8aa53d5e37647223722f7f7c/src/stats.jl#L60) (constructeur, attribut, type).


In [3]:
using SolverCore

In [4]:
? GenericExecutionStats

ErrorException: syntax: invalid identifier name "?"

- Les arguments dans les fonctions. Lire attentivement [la documentation Julia sur les fonctions](https://docs.julialang.org/en/v1/manual/functions/) pour comprendre l'utilisation des `Optional Arguments` et des `Keywords Arguments`. Ce type d'arguments est très utile dans nos applictions où les solveurs dépendent de paramètre dont on peut fixer des valeurs par défaut.

## Exercice 1: Méthode BFGS avec mémoire limitée (L-BFGS)

Le but de cet exercice est d'implémenter la méthode BFGS à mémoire limitée vue en cours en utilisant les `InverseLBFGSOperator` du package `LinearOperators.jl`. Il y a aussi un petit exemple dans la documentation du package [LinearOperators.jl/dev/tutorial/#Limited-memory-BFGS-and-SR1](https://juliasmoothoptimizers.github.io/LinearOperators.jl/dev/tutorial/#Limited-memory-BFGS-and-SR1).

In [5]:
using LinearOperators

In [6]:
? InverseLBFGSOperator

ErrorException: syntax: invalid identifier name "?"

ce qui est important dans ce type de méthode est:
- le paramètre mémoire
- la mise à jour de l'opérateur avec la fonction `push!`
- si on a pas une direction de descente, alors on skip
- recherche linéaire d'Armijo

In [7]:
? LinearOperators.push!

ErrorException: syntax: invalid identifier name "?"

In [8]:
function armijo(xk, dk, fk, gk, slope, nlp :: AbstractNLPModel; τ1 = 1.0e-4, t_update = 1.5)
  t = 1.0
  fk_new = obj(nlp, xk + dk) # t = 1.0
  while fk_new > fk + τ1 * t * slope
    t /= t_update
    fk_new = obj(nlp, xk + t * dk)
  end
  return t, fk_new
end

armijo (generic function with 1 method)

In [9]:
function limited_bfgs(nlp      :: AbstractNLPModel;
                      x        :: AbstractVector = nlp.meta.x0,
                      atol     :: Real = √eps(eltype(x)), 
                      rtol     :: Real = √eps(eltype(x)),
                      max_eval :: Int = -1,
                      max_time :: Float64 = 30.0,
                      f_min    :: Float64 = -1.0e16,
                      verbose  :: Bool = true,
                      mem      :: Int = 5)
  start_time = time()
  elapsed_time = 0.0
  s=0
  y=0
  T = eltype(x)
  n = nlp.meta.nvar

  xt = zeros(T, n)
  ∇ft = zeros(T, n)

  f = obj(nlp, x)
  ∇f = grad(nlp, x)
#################################################
  H = InverseLBFGSOperator(T,n, ;mem=5, scaling=true)
#################################################

  ∇fNorm = norm(∇f) #nrm2(n, ∇f)
  ϵ = atol + rtol * ∇fNorm
  iter = 0

  @info log_header([:iter, :f, :dual, :slope, :bk], [Int, T, T, T, T],
                   hdr_override=Dict(:f=>"f(x)", :dual=>"‖∇f‖", :slope=>"∇fᵀd"))

  optimal = ∇fNorm ≤ ϵ
  unbdd = f ≤ f_min
  tired = neval_obj(nlp) > max_eval ≥ 0 || elapsed_time > max_time
  stalled = false
  status = :unknown

  while !(optimal || tired || stalled || unbdd)

#################################################
    d = -H*∇f
#################################################
    slope = dot(d, ∇f)
    if slope ≥ 0
      @error "not a descent direction" slope
      status = :not_desc
      stalled = true
      continue
    end

    # Perform improved Armijo linesearch.
    t, ft = armijo(x, d, f, ∇f, slope, nlp)
        
    @info log_row(Any[iter, f, ∇fNorm, slope, t])

    # Update L-BFGS approximation.
    s = t*d
    xt = x + t * d
    ∇ft = grad(nlp, xt) # grad!(nlp, xt, ∇ft)
    y = ∇ft - ∇f
#################################################
    LinearOperators.push!(H, s, y)
#################################################

    # Move on.
    x = xt
    f = ft
    ∇f = ∇ft

    ∇fNorm = norm(∇f) #nrm2(n, ∇f)
    iter = iter + 1

    optimal = ∇fNorm ≤ ϵ
    unbdd = f ≤ f_min
    elapsed_time = time() - start_time
    tired = neval_obj(nlp) > max_eval ≥ 0 || elapsed_time > max_time
  end
  @info log_row(Any[iter, f, ∇fNorm])

  if optimal
    status = :first_order
  elseif tired
    if neval_obj(nlp) > max_eval ≥ 0
      status = :max_eval
    elseif elapsed_time > max_time
      status = :max_time
    end
  elseif unbdd
        status = :unbounded
  end

  return GenericExecutionStats(
        nlp,
        status=status,
        solution=x,
        objective=f,
        dual_feas=∇fNorm,
        iter=iter,
        elapsed_time=elapsed_time,
    )
end

limited_bfgs (generic function with 1 method)

In [10]:
#Unit/Validation Tests
# Réaliser un test unitaire
using Pkg; Pkg.add("Test"); Pkg.add("Logging")

    Updating registry at `C:\Users\Ulrizpascuit\.julia\registries\General.toml`


   Resolving package versions...


  No Changes to `C:\Users\Ulrizpascuit\Desktop\MTH8408\MTH8408-Hiv24\lab3\Project.toml`
  No Changes to `C:\Users\Ulrizpascuit\Desktop\MTH8408\MTH8408-Hiv24\lab3\Manifest.toml`

   Resolving package versions...


  No Changes to `C:\Users\Ulrizpascuit\Desktop\MTH8408\MTH8408-Hiv24\lab3\Project.toml`
  No Changes to `C:\Users\Ulrizpascuit\Desktop\MTH8408\MTH8408-Hiv24\lab3\Manifest.toml`

In [11]:

using Logging, Test
stats = with_logger(NullLogger()) do 
    limited_bfgs(himmelblau) 
end
@test stats.status == :first_order
@test stats.solution ≈ [3.584428266659278, -1.8481265666485827] atol = 1e-6
@show (stats.status, stats.solution)

(stats.status, stats.solution) = (:first_order, [3.584428266659278, -1.8481265666485827])


(:first_order, [3.584428266659278, -1.8481265666485827])

### Bonus:

- Compare l'implémentation de `limited_bfgs` avec la fonction `lbfgs` qui est disponible dans `JSOSolvers.jl`.
- On veut pouvoir tester "facilement" plusieurs valeurs de $\tau$ et du paramètre de mise à jour dans `armijo` sur les problèmes tests. Comment modifier le code pour que ça soit possible?

On peut mesurer deux executions de fonctions Julia grâce aux fonctions de `BenchmarkTools.jl`:

In [12]:
using BenchmarkTools

In [13]:
? @time

ErrorException: syntax: invalid identifier name "?"

## Exercice 2: NewtonCG

Le but de cet exercice est d'adapter les méthodes de Newton de façon à résoudre le système linéaire avec une méthode itérative de type gradient conjugué comme suit ($B_k$ représente la matrice hessienne):

<img src="LineSearchNewtonCG.png" width=600 height=600 />

In [14]:
function cg_optim(H, ∇f)
    #setup the tolerance:
    n∇f = norm(∇f)
    #####################################
    ϵk = min(0.5, sqrt(n∇f))*n∇f
    #################################### 
    n = length(∇f)
    z = zeros(n)
    r = ∇f
    d = -r
    
    j = 0
    while norm(r) ≥ ϵk && j < 3 * n
     ###############################################
        if dot(d, H * d) ≤ 0
            if j == 0
                p = -∇f
            else
                p = z
            end
        end
     ##############################################
        α = r' * r / (d' * H * d)
     ##############################################        
        z += α * d
        nrr2 = dot(r, r)
        r += α * H * d
     ##############################################
        β  = r' * r / nrr2
     ##############################################
        d  = -r + β * d
        j += 1
   end
   return z
end

cg_optim (generic function with 1 method)

In [15]:
matrice def posi 4x4
resoudre Hz=gradf

ErrorException: syntax: extra token "def" after end of expression

Ce qui est important ici est qu'on a pas besoin de stocker/évaluer la matrice hessienne entière mais simplement le produit entre la hessienne et un vecteur. Pour un `NLPModels` on utilise:

In [16]:
? NLPModels.hprod

ErrorException: syntax: invalid identifier name "?"

In [17]:
? NLPModels.hess_op

ErrorException: syntax: invalid identifier name "?"

In [18]:
function armijo_Newton_cg(nlp      :: AbstractNLPModel;
                          x        :: AbstractVector = nlp.meta.x0,
                          atol     :: Real = √eps(eltype(x)), 
                          rtol     :: Real = √eps(eltype(x)),
                          max_eval :: Int = -1,
                          max_time :: Float64 = 30.0,
                          f_min    :: Float64 = -1.0e16)
  start_time = time()
  elapsed_time = 0.0

  T = eltype(x)
  n = nlp.meta.nvar

  f = obj(nlp, x)
  ∇f = grad(nlp, x)
#################################################
  H = NLPModels.hess_op(nlp,x)
#################################################

  ∇fNorm = norm(∇f) #nrm2(n, ∇f)
  ϵ = atol + rtol * ∇fNorm
  iter = 0

  @info log_header([:iter, :f, :dual, :slope, :bk], [Int, T, T, T, T],
                   hdr_override=Dict(:f=>"f(x)", :dual=>"‖∇f‖", :slope=>"∇fᵀd"))

  optimal = ∇fNorm ≤ ϵ
  unbdd = f ≤ f_min
  tired = neval_obj(nlp) > max_eval ≥ 0 || elapsed_time > max_time
  stalled = false
  status = :unknown

  while !(optimal || tired || stalled || unbdd)
        
    d = cg_optim(H, ∇f)
        
    slope = dot(d, ∇f)
    if slope ≥ 0
      @error "not a descent direction" slope
      status = :not_desc
      stalled = true
      continue
    end

    # Perform improved Armijo linesearch.
    t, f = armijo(x, d, f, ∇f, slope, nlp)
        
    @info log_row(Any[iter, f, ∇fNorm, slope, t])

    # Update L-BFGS approximation.
    x += t * d
    ∇f = grad(nlp, x)
#################################################
    H = NLPModels.hess_op(nlp,x)
#################################################

    ∇fNorm = norm(∇f) #nrm2(n, ∇f)
    iter = iter + 1

    optimal = ∇fNorm ≤ ϵ
    unbdd = f ≤ f_min
    elapsed_time = time() - start_time
    tired = neval_obj(nlp) > max_eval ≥ 0 || elapsed_time > max_time
  end
  @info log_row(Any[iter, f, ∇fNorm])

  if optimal
    status = :first_order
  elseif tired
    if neval_obj(nlp) > max_eval ≥ 0
      status = :max_eval
    elseif elapsed_time > max_time
      status = :max_time
    end
  elseif unbdd
        status = :unbounded
  end

  return GenericExecutionStats(nlp, status = status, solution=x, objective=f, dual_feas=∇fNorm,
                               iter=iter, elapsed_time=elapsed_time)
end

armijo_Newton_cg (generic function with 1 method)

In [19]:
#Unit/Validation Tests
# Réaliser un test unitaire
stats = with_logger(NullLogger()) do 
    armijo_Newton_cg(pb_du_cours, x = [1.5, .5]) 
end
@test stats.status == :first_order
@test stats.solution ≈ [1., 0.] atol = 1e-6
@show (stats.status, stats.solution)

(stats.status, stats.solution) = (:first_order, [1.000000000013107, 6.553559582523889e-12])


(:first_order, [1.000000000013107, 6.553559582523889e-12])

## Comment préparer un benchmark

On veut maintenant pouvoir réaliser un benchmark de plusieurs solveurs. Pour comparer les algorithmes, il nous faut une collection de problèmes tests et on va utiliser `OptimizationProblems.jl`.

In [40]:

using ADNLPModels
using OptimizationProblems, OptimizationProblems.ADNLPProblems
using LinearAlgebra
using SolverCore, SolverBenchmark
using ADNLPModels, NLPModels
using OptimizationProblems, OptimizationProblems.ADNLPProblems
using JSOSolvers

first_order(df) = df.status .== :first_order
unbounded(df) = df.status .== :unbounded
solved(df) = first_order(df) .| unbounded(df)
solved(ges)

n = 20
solvers = solvers = Dict(
  :lbfgs => model -> lbfgs(model, mem=5),
  :armijo_Newton_cg  => model -> armijo_Newton_cg(model),
  # :trunk => model -> trunk(model),
)

ad_problems = (eval(Meta.parse(problem))(;n) for problem ∈ OptimizationProblems.meta[!, :name])

stats = bmark_solvers(
  solvers, ad_problems,
  skipif=prob -> (!unconstrained(prob) || get_nvar(prob) > 100 || get_nvar(prob) < 5),
)

costnames = ["time", "obj + grad + hess"]
costs = [
  df -> .!solved(df) .* Inf .+ df.elapsed_time,
  df -> .!solved(df) .* Inf .+ df.neval_obj .+ df.neval_grad .+ df.neval_hess,
]

using Plots
gr()

profile_solvers(stats, costs, costnames)


In [ ]:
cols = [:id, :name, :nvar, :objective, :dual_feas, :neval_obj, :neval_grad, :neval_hess, :iter, :elapsed_time, :status]
header = Dict(
  :nvar => "n",
  :objective => "f(x)",
  :dual_feas => "‖∇f(x)‖",
  :neval_obj => "# f",
  :neval_grad => "# ∇f",
  :neval_hess => "# ∇²f",
  :elapsed_time => "t",
)

for solver ∈ keys(solvers)
  pretty_stats(stats[solver][!, cols], hdr_override=header)
end